In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
import yfinance as yf

In [4]:
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings("ignore")

In [5]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_rows", 100)

In [6]:
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

In [7]:
config_params = {
    "Tickers": [
        # Technology
        "AAPL",
        "MSFT",
        "GOOGL",
        "AMZN",
        "TSLA",
        "NVDA",
        "META",
        # Financial
        "BRK-B",
        "JPM",
        "V",
        # Healthcare
        "JNJ",
        "LLY",
        "UNH",
        # Consumer Discretionary
        "PG",
        "WMT",
        "COST",
        # Energy
        "XOM",
        "CVX",
        # Benchmark
        "SPY",
    ],
    "start_date": "2015-01-01",
    "end_date": "2026-01-01",
    "risk_free_rate": 4.18,
    "rebalance_frequency": "quarterly",
    "transaction_cost": 0.0015,
    "train_test_split": 0.8,
    "validation_split": 0.2,
    "radom_seed": 42,
}

In [8]:
def download_data(tickers, start_date, end_date, progress=True):
    print("Downloading stock data from Yahoo Finance...")
    print(f"Tickers: {', '.join(tickers)}")

    try:
        data = yf.download(
            tickers=tickers,
            start=start_date,
            end=end_date,
            progress=progress,
            group_by="tickers",
            auto_adjust=True,
        )

        if data.empty:
            raise ValueError(
                "No data downloaded. Please check the tickers and date range."
            )
        return data

    except Exception as e:
        print(f"Error downloading data: {str(e)}")
        return None

In [9]:
def process_raw_data(raw_data, tickers):
    if len(tickers) == 1:
        prices = raw_data["Close"].to_frame()
        prices.columns = tickers
        volume = raw_data["Volume"].to_frame()
        volume.columns = tickers
    else:
        prices = pd.DataFrame()
        volume = pd.DataFrame()

        for ticker in tickers:
            try:
                if isinstance(raw_data.columns, pd.MultiIndex):
                    prices[ticker] = raw_data[ticker]["Close"]
                    volume[ticker] = raw_data[ticker]["Volume"]
                else:
                    prices[ticker] = raw_data[ticker]
                    volume[ticker] = raw_data[ticker]
            except KeyError:
                print(f"Warning: No data found for ticker {ticker}. Skipping.")

    returns = prices.pct_change()
    returns = returns.dropna()

    print("Processed data:")
    print(f"Prices shape: {prices.shape}")
    print(f"Returns shape: {returns.shape}")
    print(f"Missing values in prices: {prices.isna().sum().sum()}")
    print(f"Missing values in returns: {returns.isna().sum().sum()}")

    return {"prices": prices, "returns": returns, "volume": volume}


In [10]:
raw_data = download_data(
    tickers=config_params["Tickers"],
    start_date=config_params["start_date"],
    end_date=config_params["end_date"],
)

raw_data

Tickers: AAPL, MSFT, GOOGL, AMZN, TSLA, NVDA, META, BRK-B, JPM, V, JNJ, LLY, UNH, PG, WMT, COST, XOM, CVX, SPY


[*********************100%***********************]  19 of 19 completed


Ticker           BRK-B                                               \
Price             Open        High         Low       Close   Volume   
Date                                                                  
2015-01-02  151.500000  151.600006  148.500000  149.169998  3436400   
2015-01-05  148.809998  149.000000  146.779999  147.000000  4168800   
2015-01-06  147.639999  148.529999  146.110001  146.839996  4116100   
2015-01-07  147.940002  149.139999  147.649994  148.880005  4159100   
2015-01-08  150.600006  151.369995  150.509995  151.369995  4282100   
...                ...         ...         ...         ...      ...   
2025-12-24  500.290009  501.510010  499.250000  501.339996  2000200   
2025-12-26  500.450012  501.559998  496.829987  498.299988  2247600   
2025-12-29  499.200012  501.500000  497.209991  501.049988  3056200   
2025-12-30  500.980011  505.109985  500.329987  503.709991  2897100   
2025-12-31  503.920013  505.890015  502.480011  502.649994  3105200   

Ticker            MSFT                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2015-01-02   39.773201   40.421027   39.670914   39.858440  27913900   
2015-01-05   39.526007   39.832873   39.423719   39.491913  39673900   
2015-01-06   39.534528   39.849917   38.818508   38.912273  36447900   
2015-01-07   39.193576   39.602730   38.775899   39.406677  29114100   
2015-01-08   39.849928   40.702333   39.824357   40.565948  29645200   
...                ...         ...         ...         ...       ...   
2025-12-24  485.679993  489.160004  484.829987  488.019989   5855900   
2025-12-26  486.709991  488.119995  485.959991  487.709991   8842200   
2025-12-29  484.859985  488.350006  484.179993  487.100006  10893400   
2025-12-30  485.929993  489.679993  485.500000  487.480011  13944500   
2025-12-31  487.839996  488.140015  483.299988  483.619995  15601600   

Ticker            NVDA                                                 \
Price             Open        High         Low       Close     Volume   
Date                                                                    
2015-01-02    0.483011    0.486611    0.475333    0.483011  113680000   
2015-01-05    0.483011    0.484451    0.472694    0.474853  197952000   
2015-01-06    0.475573    0.476053    0.459977    0.460456  197764000   
2015-01-07    0.463816    0.467895    0.457817    0.459257  321808000   
2015-01-08    0.464536    0.479412    0.464296    0.476533  283780000   
...                ...         ...         ...         ...        ...   
2025-12-24  187.940002  188.910004  186.589996  188.610001   65528500   
2025-12-26  189.919998  192.690002  188.000000  190.529999  139740300   
2025-12-29  187.710007  188.759995  185.910004  188.220001  120006100   
2025-12-30  188.240005  188.990005  186.929993  187.539993   97687300   
2025-12-31  189.570007  190.559998  186.490005  186.500000  120100500   

Ticker             JPM                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2015-01-02   46.280394   46.860945   46.198521   46.511127  12600000   
2015-01-05   46.191082   46.354826   44.829017   45.067192  20100600   
2015-01-06   45.134177   45.216050   43.429736   43.898643  29074100   
2015-01-07   44.575957   44.575957   43.667913   43.965633  23843200   
2015-01-08   44.635517   45.327713   44.635517   44.948120  16971100   
...                ...         ...         ...         ...       ...   
2025-12-24  324.774998  328.508159  324.038331  327.691864   4289300   
2025-12-26  327.632124  329.374265  325.073687  326.437531   4158300   
2025-12-29  325.531610  326.298141  322.077190  322.296204   8635300   
2025-12-30  323.351455  323.480876  321.041865  321.967712   7904300   
2025-12-31  322.883547  323.421130  3

In [11]:
processed_data = process_raw_data(raw_data, config_params["Tickers"])

Processed data:
Prices shape: (2766, 19)
Returns shape: (2765, 19)
Missing values in prices: 0
Missing values in returns: 0


In [12]:
processed_data

{'prices':                   AAPL        MSFT       GOOGL        AMZN        TSLA  \
 Date                                                                     
 2015-01-02   24.237551   39.858440   26.278946   15.426000   14.620667   
 2015-01-05   23.554745   39.491913   25.778225   15.109500   14.006000   
 2015-01-06   23.556961   38.912273   25.142036   14.764500   14.085333   
 2015-01-07   23.887281   39.406677   25.068092   14.921000   14.063333   
 2015-01-08   24.805080   40.565948   25.155434   15.023000   14.041333   
 ...                ...         ...         ...         ...         ...   
 2025-12-24  273.809998  488.019989  314.089996  232.380005  485.399994   
 2025-12-26  273.399994  487.709991  313.510010  232.520004  475.190002   
 2025-12-29  273.760010  487.100006  313.559998  232.070007  459.640015   
 2025-12-30  273.079987  487.480011  313.850006  232.529999  454.429993   
 2025-12-31  271.859985  483.619995  313.000000  230.820007  449.720001   
 
             

In [13]:
prices = processed_data["prices"]
returns = processed_data["returns"]
volume = processed_data["volume"]

In [ ]:
def summary_stats(returns, prices, risk_free_rate):
    """Calculate summary statistics for given returns and prices DataFrames."""

    stats = pd.DataFrame(index=returns.columns)

    # Annualized Return
    avg_daily_return = returns.mean()
    stats["annualized_return"] = ((1 + avg_daily_return) ** 252) - 1

    # Annualized Volatility
    daily_volatility = returns.std()
    stats["annualized_volatility"] = daily_volatility * np.sqrt(252)

    # Sharpe Ratio
    stats["sharpe_ratio"] = (
        stats["annualized_return"] - (risk_free_rate / 100)
    ) / stats["annualized_volatility"]

    # Max Drawdown
    mdd = []

    for ticker in prices.columns:
        running_peak = prices[ticker].cummax()
        drawdown = (prices[ticker] - running_peak) / running_peak
        mdd.append(drawdown.min())

    stats["max_drawdown"] = mdd

    stats["current_price"] = prices.iloc[-1]
    stats["start_price"] = prices.iloc[0]
    stats["total_returns"] = (prices.iloc[-1] - prices.iloc[0])/prices.iloc[0]
    
    for col in ["annualized_return", "annualized_volatility", "max_drawdown", "total_returns"]
        stats[col] = stats[col].apply(lambda x: f"{x*100:.2f}%")

    stats["sharpe_ratio"] = stats["sharpe_ratio"].apply(lambda x: f"{x:.3f}%")
    stats["current_price"] = stats["current_price"].apply(lambda x: f"{x:.2f}%")
    stats["start_price"] = stats["start_price"].apply(lambda x: f"{x:.2f}%")

    return stats

In [45]:
summary_stats(
    returns=returns, prices=prices, risk_free_rate=config_params["risk_free_rate"]
)

,annualized_return,annualized_volatility,sharpe_ratio,max_drawdown
AAPL,0.299287,0.288458,0.892631,-0.385159
MSFT,0.301520,0.268881,0.965929,-0.371485
GOOGL,0.306320,0.288416,0.917148,-0.443200
AMZN,0.350458,0.329283,0.937364,-0.561453
TSLA,0.612789,0.577511,0.988707,-0.736322
NVDA,0.934711,0.486920,1.833795,-0.663351
META,0.304447,0.375319,0.699796,-0.767361
BRK-B,0.138016,0.192718,0.499257,-0.295699
JPM,0.237049,0.271490,0.719177,-0.436265
V,0.207156,0.242595,0.681614,-0.363649


In [14]:
prices

,AAPL,MSFT,GOOGL,AMZN,TSLA,NVDA,META,BRK-B,JPM,V,JNJ,LLY,UNH,PG,WMT,COST,XOM,CVX,SPY
Date,,,,,,,,,,,,,,,,,,,
2015-01-02,24.237551,39.858440,26.278946,15.426000,14.620667,0.483011,77.905800,149.169998,46.511127,61.340973,76.955582,57.476723,84.480469,66.518333,23.219233,114.136017,57.916908,70.209572,170.589615
2015-01-05,23.554745,39.491913,25.778225,15.109500,14.006000,0.474853,76.654556,147.000000,45.067192,59.986938,76.418076,56.910679,83.088959,66.202065,23.151657,112.838394,56.332199,67.403168,167.508804
2015-01-06,23.556961,38.912273,25.142036,14.764500,14.085333,0.460456,75.621758,146.839996,43.898643,59.600414,76.042603,57.197800,82.921303,65.900520,23.330061,114.329445,56.032700,67.371971,165.931000
2015-01-07,23.887281,39.406677,25.068092,14.921000,14.063333,0.459257,75.621758,148.880005,43.965633,60.398960,77.721298,56.795818,83.767975,66.246216,23.949059,116.320251,56.600468,67.315857,167.998764
2015-01-08,24.805080,40.565948,25.155434,15.023000,14.041333,0.476533,77.637665,151.369995,44.948120,61.209034,78.332390,58.141258,87.766495,67.003746,24.454529,117.319687,57.542580,68.856239,170.979874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-24,273.809998,488.019989,314.089996,232.380005,485.399994,188.610001,667.549988,501.339996,327.691864,355.140015,207.779999,1076.979980,327.579987,144.490005,111.610001,871.859985,119.220001,150.500000,690.380005
2025-12-26,273.399994,487.709991,313.510010,232.520004,475.190002,190.529999,663.289978,498.299988,326.437531,355.000000,207.630005,1077.750000,331.829987,144.740005,111.739998,873.349976,119.110001,150.020004,690.309998
2025-12-29,273.760010,487.100006,313.559998,232.070007,459.640015,188.220001,658.690002,501.049988,322.296204,354.609985,207.559998,1078.729980,328.940002,144.570007,112.529999,867.840027,120.529999,150.990005,687.849976


In [15]:
prices["AAPL"]

Date
2015-01-02     24.237551
2015-01-05     23.554745
2015-01-06     23.556961
2015-01-07     23.887281
2015-01-08     24.805080
                 ...    
2025-12-24    273.809998
2025-12-26    273.399994
2025-12-29    273.760010
2025-12-30    273.079987
2025-12-31    271.859985
Name: AAPL, Length: 2766, dtype: float64

In [19]:
prices["AAPL"].iloc[-1]

np.float64(271.8599853515625)

In [20]:
prices["AAPL"].iloc[0]

np.float64(24.237550735473633)

In [22]:
current_price = prices["AAPL"].iloc[-1]
start_price = prices["AAPL"].iloc[0]
total_return = (current_price - start_price) / start_price
total_return

np.float64(10.21647926882617)